![alt text](https://i.imgur.com/1WaY7aA.png)



---



---



# Lab 2.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [2]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [3]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code of the response:

In [4]:
print(response)

<Response [200]>


We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [5]:
parameters = {"lat": -33.87, "lon": 151.21}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [6]:
response = requests.get("http://api.open-notify.org/iss-pass.json", params = parameters)

Print the response header:

In [7]:
print(response.headers)

{'Server': 'nginx/1.10.3', 'Date': 'Tue, 16 Jul 2019 04:34:37 GMT', 'Content-Type': 'application/json', 'Content-Length': '521', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}


Print the content of the response (the data that the server returned):

In [8]:
print(response.content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1563250204, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 570, \n      "risetime": 1563253483\n    }, \n    {\n      "duration": 659, \n      "risetime": 1563259261\n    }, \n    {\n      "duration": 401, \n      "risetime": 1563265155\n    }, \n    {\n      "duration": 605, \n      "risetime": 1563313443\n    }, \n    {\n      "duration": 631, \n      "risetime": 1563319223\n    }\n  ]\n}\n'


Note that this is a Python byte string:

In [9]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [10]:
print(response.headers["content-type"])

application/json


In [11]:
print(type(response.headers["content-type"]))

<class 'str'>


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [12]:
overheads = response.json()
print(overheads)

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1563250204, 'latitude': -33.87, 'longitude': 151.21, 'passes': 5}, 'response': [{'duration': 570, 'risetime': 1563253483}, {'duration': 659, 'risetime': 1563259261}, {'duration': 401, 'risetime': 1563265155}, {'duration': 605, 'risetime': 1563313443}, {'duration': 631, 'risetime': 1563319223}]}


What kind of object did this give us?

In [13]:
print(type(overheads))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [28]:
#First, extract the passes value to a separate dict:
passes = overheads['response']
print(passes)

[{'duration': 570, 'risetime': 1563253483}, {'duration': 659, 'risetime': 1563259261}, {'duration': 401, 'risetime': 1563265155}, {'duration': 605, 'risetime': 1563313443}, {'duration': 631, 'risetime': 1563319223}]


Now extract the `risetime` strings into an array called `srisetimes`:

In [33]:
srisetimes = [xpass['risetime'] for xpass in passes]


In [34]:
print(srisetimes)

[1563253483, 1563259261, 1563265155, 1563313443, 1563319223]


```
datetime.fromtimestamp(str)
```

Convert these to an array of Python `datetime` values called `risetimes`:

In [35]:
#ANSWER:
risetimes = [datetime.fromtimestamp(xpass['risetime']) for xpass in passes] 

In [36]:
for risetime in risetimes:
    print(risetime.strftime('%d/%m/%y %I:%M'))

16/07/19 03:04
16/07/19 04:41
16/07/19 06:19
17/07/19 07:44
17/07/19 09:20


Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



Here is an endpoint that tells us who is onboard:

In [37]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [38]:
astros = response.json()
print(astros)
print(astros["number"])
[print(astronaut['name']) for astronaut in astros['people']]

{'people': [{'craft': 'ISS', 'name': 'Alexey Ovchinin'}, {'craft': 'ISS', 'name': 'Nick Hague'}, {'craft': 'ISS', 'name': 'Christina Koch'}], 'number': 3, 'message': 'success'}
3
Alexey Ovchinin
Nick Hague
Christina Koch


[None, None, None]

## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [39]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code, 
    '''
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

In [40]:
def handleResponse(response, verbose = False):
    if response.status_code == 200:
        if verbose:
            print('Response = 200: OK')
        fail = False
    else:
        if verbose:
            print('Response = ' + str(response.status_code) + ': ' + '(TODO: output of lookup function here)')
        fail = True
    return(fail, response.status_code)

2. Test your response handler on some correct and incorrect API calls.

In [41]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-pass.json")
handleResponse(response, True)[0]

Response = 400: (TODO: output of lookup function here)


True

>

>

>



---



---


> > > > > > > > >© 2019 Data Science Institute of Australia


---



---

